In [24]:
import sys
sys.path.append("..")


%reload_ext autoreload
%autoreload 2

import pandas as pd
import chess
import chess.pgn
import io
from utils import load_games_from_pgn_file, get_nags, get_comments

from common.utils import save_to_sql

In [2]:
import os

def get_pgn_files(path):
    pgn_files = []
    for filename in os.listdir(path):
        if filename.endswith(".pgn"):
            pgn_files.append(os.path.join(path, filename))
    return pgn_files

In [3]:
books_directory = "../../data/libgen_books/Categories/winner_marked_or_last_move/"

In [4]:
games = []

for path in get_pgn_files(books_directory):
    games += load_games_from_pgn_file(path)

error during pgn parsing
Traceback (most recent call last):
  File "/home/kamil/.local/lib/python3.10/site-packages/chess/pgn.py", line 1685, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "/home/kamil/.local/lib/python3.10/site-packages/chess/pgn.py", line 1059, in parse_san
    return board.parse_san(san)
  File "/home/kamil/.local/lib/python3.10/site-packages/chess/__init__.py", line 3039, in parse_san
    raise ValueError(f"illegal san: {san!r} in {self.fen()}")
ValueError: illegal san: 'Be3' in r1bqkbnr/pp1ppppp/2n5/2p5/4P3/2N5/PPPP1PPP/R1BQKBNR w KQkq - 2 3
error during pgn parsing
Traceback (most recent call last):
  File "/home/kamil/.local/lib/python3.10/site-packages/chess/pgn.py", line 1685, in read_game
    move = visitor.parse_san(board_stack[-1], token)
  File "/home/kamil/.local/lib/python3.10/site-packages/chess/pgn.py", line 1059, in parse_san
    return board.parse_san(san)
  File "/home/kamil/.local/lib/python3.10/site-packages/chess/__init_

In [7]:
nags = []
for game in games:
    nags += get_nags(game)

len(nags)

24310

In [6]:
comments = []
for game in games:
    comments += get_comments(game)

print(len(comments))

17431


In [8]:
moves_df = pd.DataFrame(nags, columns=["position", "move", "sentiment"]).astype({"position": "str", "move": str, "sentiment": int})
moves_df

,position,move,sentiment
0,rn2rbk1/1b3p1p/pq4p1/1p1pN3/5P2/2NB1Q2/PP4PP/3...,e5f7,1
1,rn2rbk1/1b3N1p/p5p1/1p1p4/5P2/2NBqQ2/PP4PP/3R1...,f3g4,1
2,r3rbk1/1b3N1p/p4nB1/1p1p4/5PQ1/2N1q3/PP4PP/3R1...,g6h7,1
3,6k1/4Q2p/8/p3Np2/1p1Pbq2/1P6/P1P3r1/1KR5 b - -...,e4c2,1
4,1kr5/3p1pR1/p3p2p/1p2PP1P/1q1N4/bP5r/P1P5/1K1R...,h3h1,1
...,...,...,...
24305,8/1kp1PQ2/1pb5/p7/2P5/1P6/1KP1rp2/8 b - - 2 47,e2e7,1
24306,8/1kp2Q2/1pb5/8/1PP1r3/8/3K4/8 b - - 0 57,b6b5,1
24307,8/1kp5/2b5/2Q5/1P2r3/8/3K4/8 b - - 3 60,e4e6,1
24308,8/1kp2Q2/2b5/1p6/1PP1r3/8/3K4/8 w - - 0 58,c4c5,0


In [9]:
len(moves_df[moves_df.sentiment == 1]), len(moves_df[moves_df.sentiment == 0])

(20089, 4221)

In [10]:
def get_color_moves(game: chess.pgn.GameNode, turn = False):
    result = []

    if game.parent and game.turn() == turn:
        result.append((game.parent.board().fen(), game.move))

    for node in game.variations:
        result += get_color_moves(node, turn)

    return result

def get_all_moves(game: chess.pgn.GameNode):
    result = []

    if game.parent:
        result.append((game.parent.board().fen(), game.move))

    for node in game.variations:
        result += get_all_moves(node)

    return result

In [11]:
game = games[1]
print(game)

[Event "I a Georgia (ch)"]
[Site "?"]
[Date "2022.??.??"]
[Round "?"]
[White "Sanikidze, Tornike"]
[Black "Gagunashvili, Merab"]
[Result "0-1"]
[Annotator "Arsovic,Goran"]
[BlackElo "2573"]
[EventDate "2022.??.??"]
[FEN "6k1/4Q2p/8/p3Np2/1p1Pbq2/1P6/P1P3r1/1KR5 b - - 0 1"]
[PlyCount "3"]
[SetUp "1"]
[Source "Chess Informant"]
[SourceDate "2022.03.24"]
[SourceQuality "2"]
[SourceTitle "151 Combinations Section"]
[SourceVersion "1"]
[SourceVersionDate "2022.03.24"]
[WhiteElo "2464"]

{ Diagram [#] } 1... Bxc2+ $1 2. Rxc2 Rg1+ 0-1


In [16]:
def get_winner_moves(game: chess.pgn.GameNode):
    try:
        result = game.headers.get('Result', '*')
        
        if result == '1-0':
            return get_color_moves(game, chess.WHITE)
        elif result == '0-1':
            return get_color_moves(game, chess.BLACK)
        elif result == '1/2-1/2':
            return get_all_moves(game)
        
        last_turn = game.end().turn()

        return get_color_moves(game, last_turn)
    except:
        return []

In [17]:
game_str = """
[Event "#4908 Chess: 5334 Problems, Combination"]
[Site "Vinn de Mar 1947"]
[Date "1947.??.??"]
[Round "?"]
[White "Rossetto"]
[Black "Stahlberg"]
[Result "1/2-1/2"]
[ECO "C14"]
[Annotator "VB"]
[PlyCount "37"]
[EventDate "1947.??.??"]

1. e4 e6 2. d4 d5 3. Nc3 Nf6 4. Bg5 Be7 5. e5 Nfd7 6. h4 c5 7. Nb5 f6 8. Bd3 a6
9. Qh5+ Kf8 10. Rh3 axb5 11. Bh6 Qa5+ 12. Bd2 Qc7 13. Rg3 cxd4 14. Nf3 Nxe5 {
[#]} 15. Rxg7 $1 $11 h6 (15... Kxg7 $2 16. Bh6+ Kg8 17. Qe8+ $18) 16. Bh7 Kxg7
(16... Nxf3+ $2 17. Kd1 $1 $18 (17. gxf3 $2 Qe5+ $19)) 17. Qxh6+ $11 Kf7 18.
Qh5+ Kg7 (18... Kf8 $2 19. Bh6#) 19. Qh6+ $11 1/2-1/2
"""

In [18]:
good_moves = []

for game in games:
    good_moves += get_winner_moves(game)

len(good_moves)

839263

In [22]:
good_moves_df = pd.DataFrame(good_moves, columns=["position", "move"]).astype({"position": "str", "move": str})
good_moves_df['sentiment'] = 1
good_moves_df


,position,move,sentiment
0,rn2rbk1/1b3N1p/pq4p1/1p1p4/5P2/2NB1Q2/PP4PP/3R...,b6e3,1
1,rn2rbk1/1b3N1p/p5p1/1p1p4/5PQ1/2NBq3/PP4PP/3R1...,b8d7,1
2,r3rbk1/1b1n1N1p/p5p1/1p1p1P2/6Q1/2NBq3/PP4PP/3...,d7f6,1
3,r3rbk1/1b3N1p/p4nP1/1p1p4/6Q1/2NBq3/PP4PP/3R1R...,h7h5,1
4,r3rbk1/1b3N2/p4nP1/1p1p3p/7Q/2NBq3/PP4PP/3R1R1...,f8e7,1
...,...,...,...
839258,8/1kp2Q2/1pb5/p1P5/4r3/1PP5/3K4/8 b - - 0 56,b6b5,1
839259,8/1kp1PQ2/1pb5/p7/2P5/1P6/1KP1rp2/8 b - - 2 47,f2f1q,1
839260,8/1kp1PQ2/1pb5/p7/2P5/1P6/1KP1r3/5q2 w - - 0 48,f7f1,1
839261,8/1kp1P3/1pb5/p7/2P5/1P6/1KP1r3/5Q2 b - - 0 48,e2e7,1


In [23]:
books_moves = pd.concat([moves_df, good_moves_df])
books_moves

,position,move,sentiment
0,rn2rbk1/1b3p1p/pq4p1/1p1pN3/5P2/2NB1Q2/PP4PP/3...,e5f7,1
1,rn2rbk1/1b3N1p/p5p1/1p1p4/5P2/2NBqQ2/PP4PP/3R1...,f3g4,1
2,r3rbk1/1b3N1p/p4nB1/1p1p4/5PQ1/2N1q3/PP4PP/3R1...,g6h7,1
3,6k1/4Q2p/8/p3Np2/1p1Pbq2/1P6/P1P3r1/1KR5 b - -...,e4c2,1
4,1kr5/3p1pR1/p3p2p/1p2PP1P/1q1N4/bP5r/P1P5/1K1R...,h3h1,1
...,...,...,...
839258,8/1kp2Q2/1pb5/p1P5/4r3/1PP5/3K4/8 b - - 0 56,b6b5,1
839259,8/1kp1PQ2/1pb5/p7/2P5/1P6/1KP1rp2/8 b - - 2 47,f2f1q,1
839260,8/1kp1PQ2/1pb5/p7/2P5/1P6/1KP1r3/5q2 w - - 0 48,f7f1,1
839261,8/1kp1P3/1pb5/p7/2P5/1P6/1KP1r3/5Q2 b - - 0 48,e2e7,1


In [ ]:
save_to_sql(books_moves, "../../chess.db", "moves_from_books", if_exists="replace")